<a href="https://colab.research.google.com/github/DrAlexSanz/Pix2Pix-DOTCSV/blob/master/Pix2Pix_DOTCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is just for me, an implementation of the pix2pix algorithm following DotCSV's tutorial. It is very useful to get to know Tensorflow 2.0 and then I will use it for different things.

Video: [Video dotCSV](https://youtu.be/YsrMGcgfETY)

The tutorial, as he explains at the beginning, is based on this code:

[Tutorial Pix 2 Pix tensorflow 2.0](https://www.tensorflow.org/tutorials/generative/pix2pix)

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Define paths

# Root

PATH = "/content/drive/My Drive/Pix2PixFlowers"

# Input path

INPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/InputFlowers"

# Output path

OUTPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/OutputFlowers"

# Checkpoints directory

CKPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/Checkpoints"